In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

import nltk
import time
import string
import numpy as np
import pandas as pd
import html,re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import *
from sklearn.model_selection import train_test_split

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

from google.colab import drive
drive.mount('/content/drive/')

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend('rt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#Sorting out the comments

TwComments = pd.read_csv ('/content/drive/My Drive/Extras/tweetsbitcoin.csv',delimiter=",", index_col=None)
TwComments = TwComments.dropna() 
TwComments=TwComments.drop_duplicates()
TwComments=TwComments.sort_values(['date','time'], ascending=[True,True])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



In [0]:
i = TwComments[(TwComments.date == '2017-09-30')].index
TwComments = TwComments.drop(i)
TwComments["DateTime"]=pd.to_datetime(TwComments['date'] + ' ' + TwComments['time'])
TwComments["hour"] = TwComments.DateTime.dt.hour
TwComments["day"] = TwComments.DateTime.dt.weekday_name
TwComments["DateTime"] = TwComments.DateTime.values.astype(np.int64) // 10 ** 9
TwComments["TimeDiff"]= TwComments["DateTime"] - (TwComments["DateTime"] % 86400)
TwComments = TwComments[TwComments.TimeDiff > 0]

In [0]:
startTime = int(round(time.time()*60))
tcomm=pd.DataFrame()
grouped_terms = TwComments.groupby(["TimeDiff"])

In [0]:
#This is done to combine the tweets in that hour into a list
i = 0
tweets = []
for name, group in grouped_terms:
  t = []
  for row, data in group.iterrows():
    t.append(data['tweet'])
  tweets.append(''.join(t))

In [0]:
Price = pd.read_csv ('/content/drive/My Drive/Extras/1Daybinanceprices.csv',delimiter="\t", index_col=None)

In [0]:
Price["Date"] = pd.to_datetime(Price['ClTime'], unit = 'ms')
Price = Price.sort_values(['OpTime'], ascending=True)

In [0]:
def preprocess(tweets):
    t = html.unescape(tweets)
    z = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
    t = z(t)
    tweet = ' '.join(re.sub("(@[_A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",t).split())
    return tweet

#Tokenizing the tweets

def tokenizer(tweets):
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweets.lower())).strip()
    stemmer = PorterStemmer()
    tweets = [stemmer.stem(tweet.lower()) for tweet in tweets.split()]
    return tweets

def basic_tokenizer(tweet):
    #Same as tokenize but without the stemmer
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

In [0]:
for i,t in enumerate(tweets):
  tweets[i] = tokenizer(preprocess(t))

/usr/lib/python3.6/re.py:212: FutureWarning:

split() requires a non-empty pattern match.



In [0]:
vocabulary_size = 1000000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)
data = pad_sequences(sequences, maxlen=15000)

In [0]:
Price['PriceDiff'] = Price['Close']-Price['Open']

In [0]:
price_diff = []
for p in Price['PriceDiff']:
  if p >= 0:
    price_diff.append(1)
  else:
    price_diff.append(0)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, price_diff, test_size=0.1, shuffle=False)

In [0]:
model = Sequential()
model.add(Embedding(1000000, 100, input_length=15000))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

NameError: ignored

In [0]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
328/328 [==============================] - 485s 1s/step - loss: 0.0047 - acc: 0.9970
Epoch 2/10
328/328 [==============================] - 488s 1s/step - loss: 0.0037 - acc: 1.0000
Epoch 3/10
328/328 [==============================] - 480s 1s/step - loss: 0.0030 - acc: 1.0000
Epoch 4/10
328/328 [==============================] - 492s 1s/step - loss: 0.0022 - acc: 1.0000
Epoch 5/10
328/328 [==============================] - 489s 1s/step - loss: 0.0021 - acc: 1.0000
Epoch 6/10
328/328 [==============================] - 478s 1s/step - loss: 0.0029 - acc: 1.0000
Epoch 7/10
328/328 [==============================] - 480s 1s/step - loss: 0.0014 - acc: 1.0000
Epoch 8/10
328/328 [==============================] - 491s 1s/step - loss: 0.0013 - acc: 1.0000
Epoch 9/10
328/328 [==============================] - 490s 1s/step - loss: 0.0016 - acc: 1.0000
Epoch 10/10
328/328 [==============================] - 478s 1s/step - loss: 0.0011 - acc: 1.0000


In [0]:
y = model.predict(X_test)

In [0]:
t = y
for i,z in enumerate(y):
  if z>0.4:
    t[i]=1
  else:
    t[i]=0

In [0]:
t = list(map(int, t))
print(t)
su = 0

[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]


In [0]:
print(23/37)

0.6216216216216216


In [0]:
print(y_test)

[1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1]
